# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weather_df = pd.read_csv("Results/city_weather.csv")
weather_df.dropna(inplace = True) 

weather_df

,Unnamed: 0,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country
0,0,-13.70,-76.22,70.48,60,0,8.05,3932145,Pisco,PE
1,1,-54.80,-68.30,55.40,43,75,16.11,3833367,Ushuaia,AR
2,2,66.53,66.60,20.30,96,98,9.66,1493197,Salekhard,RU
3,3,-31.95,141.43,64.40,31,2,5.66,2173911,Broken Hill,AU
4,4,-33.93,18.42,58.96,77,0,16.11,3369157,Cape Town,ZA
...,...,...,...,...,...,...,...,...,...,...
553,553,66.83,70.83,19.98,97,94,11.65,1486321,Yar-Sale,RU
554,554,61.95,159.23,30.83,88,0,5.73,2125693,Evensk,RU
555,555,42.16,-72.33,70.66,68,75,5.82,4946620,Palmer,US
556,556,56.55,76.63,40.84,69,13,7.07,1501000,Kyshtovka,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Long"]]

# Fill any NaN values and convert to float
humidity = weather_df["Humidity (%)"].astype(float)

#find min and max humidity values
min_humid = weather_df["Humidity (%)"].min()
max_humid = weather_df["Humidity (%)"].max()
print(min_humid)
print(max_humid)

7
100


In [11]:
# # comment out code so don't run all 500 cities again.  

# # Plot Heatmap
# fig1 = gmaps.figure()

# # Create heat layer
# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius=2)

# # Add layer
# fig1.add_layer(heat_layer)

# # Customize the size of the figure
# figure_layout = {
#     'width': '800px',
#     'height': '600px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig1 = gmaps.figure(layout=figure_layout)

# # Display figure
# fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
weather_df.head(0)

,Unnamed: 0,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country


In [57]:
# * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = weather_df.loc[((weather_df["Temp (F)"] > 70) & (weather_df["Temp (F)"] < 80)) 
                          & (weather_df["Wind Speed (mph)"] < 10) 
                          & (weather_df["Cloudiness (%)"] == 0), :]
ideal_df = ideal_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [67]:
# # comment out code to keep random cities list constant
# # Store into variable named `hotel_df`.
# # Add a "Hotel Name" column to the DataFrame.

# # select 10 random cities
# temp_df = ideal_df.sample(10)

#export to csv,  to keep randomly selected cities
hotel_cities = temp_df.to_csv("Results_Vpy/hotel_cities.csv")
hotel_df = pd.read_csv("Results_VPy/hotel_cities.csv")
hotel_df["Hotel Name"] = ''

In [68]:
# # Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ''
hotel_df

,Unnamed: 0,Unnamed: 0.1,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country,Hotel Name
0,485,485,-30.60,-71.20,78.84,26,0,9.15,3877918,Ovalle,CL,
1,308,308,24.63,-101.42,70.72,24,0,4.45,4013039,Concepción del Oro,MX,
2,324,324,36.96,36.20,78.01,74,0,3.00,296852,Erzin,TR,
3,462,462,26.10,-106.97,73.74,28,0,7.43,4005370,Guadalupe y Calvo,MX,
4,404,404,29.97,40.21,75.20,13,0,4.70,102527,Sakakah,SA,
5,529,529,37.31,40.74,71.60,37,0,9.17,304797,Mardin,TR,
6,180,180,-2.42,115.25,74.32,91,0,1.10,1651461,Amuntai,ID,
7,261,261,24.69,46.72,73.40,23,0,3.36,108410,Riyadh,SA,
8,302,302,31.61,34.76,73.71,67,0,7.92,293842,Kiryat Gat,IL,
9,474,474,31.35,27.25,73.40,60,0,5.82,352733,Marsá Maţrūḩ,EG,


In [86]:
# find the closest hotel to each coordinate set. note, default units for distance are metric (meters) 
from pprint import pprint
params = {"type": "lodging","keyword": "Hotel", "radius": 5000, "key": g_key} 

# use iterrows to iterate through coordinate pairs within pandas dataframe
for x,y in hotel_df.iterrows():
    lat = y["Lat"]
    long = y["Long"]
    
    #add location param to params for each iteration of coordinates
    params["location"] = f"{lat}, {long}"
    
    #establish url for each iteration of coordinates
    #basic url construction: https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
    #output is json or xml, we choose json
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response['results']
        
    try:
        hotel_df.loc[x, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
        pass
    
hotel_df

,Unnamed: 0,Unnamed: 0.1,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country,Hotel Name
0,485,485,-30.60,-71.20,78.84,26,0,9.15,3877918,Ovalle,CL,Hotel Keo
1,308,308,24.63,-101.42,70.72,24,0,4.45,4013039,Concepción del Oro,MX,Hotel Las Flores
2,324,324,36.96,36.20,78.01,74,0,3.00,296852,Erzin,TR,Hattusa Vacation Thermal Club Erzin
3,462,462,26.10,-106.97,73.74,28,0,7.43,4005370,Guadalupe y Calvo,MX,"Isabel Hotel & Suites, Guadalupe y Calvo"
4,404,404,29.97,40.21,75.20,13,0,4.70,102527,Sakakah,SA,Alnuzl Special Hotel
5,529,529,37.31,40.74,71.60,37,0,9.17,304797,Mardin,TR,Mardius Tarihi Konak
6,180,180,-2.42,115.25,74.32,91,0,1.10,1651461,Amuntai,ID,Minosa Resort Hotel
7,261,261,24.69,46.72,73.40,23,0,3.36,108410,Riyadh,SA,Four Seasons Hotel Riyadh At Kingdom Center
8,302,302,31.61,34.76,73.71,67,0,7.92,293842,Kiryat Gat,IL,Desert Gat
9,474,474,31.35,27.25,73.40,60,0,5.82,352733,Marsá Maţrūḩ,EG,Beau Site


In [88]:
#export to csv and also import as new df as a backup
hotels = hotel_df.to_csv("Results_Vpy/hotels.csv")
hotel_names_df = pd.read_csv("Results_VPy/hotels.csv")

In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
coordinates = hotel_df[["Lat", "Long"]]

In [96]:
# # Customize the size of the figure
# figure_layout = {
#     'width': '800px',
#     'height': '600px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig1 = gmaps.figure(layout=figure_layout)

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates)
# Display figure
fig1.add_layer(markers)
fig1
fig1.savefig("Results_VPy/Fig 01 - Ideal Vacation Spots", dpi=100)

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…